In [1]:
from pyspark import SparkConf, SparkContext
conf=SparkConf().setMaster("local").setAppName("FriendsByAge")
sc=SparkContext(conf=conf)

In [2]:
directory="C:\\Users\\daesi\\Downloads\\빅데이터 소스코드\\소스코드\\study_spark\\data"
filename="fakefriends.csv"

In [4]:
lines = sc.textFile(f"file:///{directory}\\{filename}")
lines.collect()[:6]

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21',
 '5,Weyoun,59,318']

# 라인 파싱 함수 생성
-각 라인을 쪼개고 파싱하는 함수를 만들어 둔다.

In [9]:
def parseLine(line):
    fields = line.split(',')
    age=int(fields[2])
    numFriends = int(fields[3])
    
    return age, numFriends

In [10]:
#직접 만든 함수를 map에서 실행할 수 있도록 콜백으로 전달한다.
rdd= lines.map(parseLine)
rdd.collect()[:5]

[(33, 385), (26, 2), (55, 221), (40, 465), (68, 21)]

# key-value RDD 생성
-친구의 age를 key로 놓고, 개수를 세기 위해 value에 1을 갖는 key-value RDD를 생성한다.

In [11]:
age_count_rdd = rdd.mapValues(lambda x : (x,1))
age_count_rdd.collect()[:5]

[(33, (385, 1)), (26, (2, 1)), (55, (221, 1)), (40, (465, 1)), (68, (21, 1))]

reduceByKey를 이용해 같은 나이 key를 갖는 데이터를 집계한다.

In [14]:
#x와 y는 각 항목을 의미한다. x는 친구의 수 y는 1
totalsByAge=age_count_rdd.reduceByKey(lambda x, y : (x[0] + y[0],x[1]+y[1]))
totalsByAge.collect()[:5]

[(33, (3904, 12)),
 (26, (4115, 17)),
 (55, (3842, 13)),
 (40, (4264, 17)),
 (68, (2696, 10))]

In [21]:
#평균 친구수 구하기
averageByAge= totalsByAge.mapValues(lambda x : x[0]/x[1])
averageByAge.collect()[:5]

[(33, 325.3333333333333),
 (26, 242.05882352941177),
 (55, 295.53846153846155),
 (40, 250.8235294117647),
 (68, 269.6)]

In [22]:
result = averageByAge.collect()
for result in result:
    print(result)

(33, 325.3333333333333)
(26, 242.05882352941177)
(55, 295.53846153846155)
(40, 250.8235294117647)
(68, 269.6)
(59, 220.0)
(37, 249.33333333333334)
(54, 278.0769230769231)
(38, 193.53333333333333)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(56, 306.6666666666667)
(43, 230.57142857142858)
(36, 246.6)
(22, 206.42857142857142)
(35, 211.625)
(45, 309.53846153846155)
(60, 202.71428571428572)
(67, 214.625)
(19, 213.27272727272728)
(30, 235.8181818181818)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(42, 303.5)
(49, 184.66666666666666)
(48, 281.4)
(50, 254.6)
(39, 169.28571428571428)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(31, 267.25)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(41, 268.55555555555554)
(44, 282.1666666666667)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(29, 215.91666666666666)
(18, 343.375)
(47, 233.22222222222

In [23]:
sc.stop()